In [2]:
import numpy as np
import resampy
from scipy.io import wavfile
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import mel_features
import vggish_params
import glob
import os
import soundtransfer
global files
import pickle
MAX_COUNT_LEN = 20000
import matplotlib.pyplot as plt

In [3]:
def waveform_to_melspec(data, sample_rate):
    """Converts audio waveform into an array of examples for VGGish.
  
    Args:
      data: np.array of either one dimension (mono) or two dimensions
        (multi-channel, with the outer dimension representing channels).
        Each sample is generally expected to lie in the range [-1.0, +1.0],
        although this is not required.
      sample_rate: Sample rate of data.
  
    Returns:
      3-D np.array of shape [num_examples, num_frames, num_bands] which represents
      a sequence of examples, each of which contains a patch of log mel
      spectrogram, covering num_frames frames of audio and num_bands mel frequency
      bands, where the frame length is vggish_params.STFT_HOP_LENGTH_SECONDS.
    """
    # Convert to mono.
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)
    # Resample to the rate assumed by VGGish.
    if sample_rate != vggish_params.SAMPLE_RATE:
        data = resampy.resample(data, sample_rate, vggish_params.SAMPLE_RATE)
  
    # Compute log mel spectrogram features.
    log_mel = mel_features.log_mel_spectrogram(
        data,
        audio_sample_rate=vggish_params.SAMPLE_RATE,
        log_offset=vggish_params.LOG_OFFSET,
        window_length_secs=vggish_params.STFT_WINDOW_LENGTH_SECONDS,
        hop_length_secs=vggish_params.STFT_HOP_LENGTH_SECONDS,
        num_mel_bins=vggish_params.NUM_MEL_BINS,
        lower_edge_hertz=vggish_params.MEL_MIN_HZ,
        upper_edge_hertz=vggish_params.MEL_MAX_HZ)
  
    # Frame features into examples.
    features_sample_rate = 1.0 / vggish_params.STFT_HOP_LENGTH_SECONDS
    example_window_length = int(round(
        vggish_params.EXAMPLE_WINDOW_SECONDS * features_sample_rate))
    example_hop_length = int(round(
        vggish_params.EXAMPLE_HOP_SECONDS * features_sample_rate))
    log_mel_examples = mel_features.frame(
        log_mel,
        window_length=example_window_length,
        hop_length=example_hop_length)
    return log_mel_examples

def get_frames(x, frame_sec, stride_sec, sr):
    frames = []
    frame_len = int(frame_sec * sr)
    stride_len = int(stride_sec * sr)

    st = 0
    en = st + frame_len
    while(en < len(x)):
        frame = x[st:en]
        frames.append(frame)
        st += stride_len
        en = st + frame_len

    return np.array(frames)

def waveform_to_timeseg(data, sample_rate):
    # Convert to mono
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)
    # Resample to the rate assumed by VGGish.
    if sample_rate != vggish_params.SAMPLE_RATE:
        data = resampy.resample(data, sample_rate, vggish_params.SAMPLE_RATE)
    timeseg = get_frames(data, 2, 1, vggish_params.SAMPLE_RATE)
    return timeseg

def wavfile_to_features(wav_file, feature_mode='mel_spec'):
    """Convenience wrapper around waveform_to_examples() for a common WAV format.
  
    Args:
      wav_file: String path to a file, or a file-like object. The file
      is assumed to contain WAV audio data with signed 16-bit PCM samples.
  
    Returns:
      See waveform_to_examples.
    """
    sr, wav_data = wavfile.read(wav_file)
    assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
    samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]
    if len(samples) == 0:
        return 0
    if feature_mode == 'mel_spec':
        features = waveform_to_melspec(samples, sr)
    elif feature_mode == 'time_seg':
        features = waveform_to_timeseg(samples, sr)
    return features

In [3]:
feature_mode = 'time_seg'

for mode in ['test']:
    parent_dir = os.path.join('ubicoustics_data', mode)
    label_name_list = os.listdir(parent_dir)
    
    for f in range(3):
        ll = len(label_name_list)
        label_list = label_name_list[int(f/3*ll):int((f+1)/3*ll)]
        print(label_list)
        features = []
        labels = []
        
        for label_name in label_list:
            class_features = []
            class_labels = []
            sub_dir = os.path.join(parent_dir, label_name)
            files = os.listdir(sub_dir)

            for fn in files:
                if fn.endswith('.wav') and not fn.startswith('.'):
                    print("Processing file", mode, label_name, fn)
                    file = os.path.join(sub_dir, fn)
                    try:
                        feat = wavfile_to_features(file, feature_mode)
                        lbl = [soundtransfer.label[label_name]] * len(feat)
                        assert len(feat) == len(lbl)
                        class_features.append(feat)
                        class_labels.append(lbl)
                    except AssertionError:
                        print('len feat = {} != len lbl = {}'.format(len(feat), len(lbl)))
                    except:
                        print('--- Error while processing')

            if len(class_features)!=0:
                features.append(class_features)
                labels.append(class_labels)
                assert len(class_features) == len(class_labels)
                print('-'*20)
                print(label_name, len(class_features))
                print()

#         np.save(np.array(features), "features/{}/{}_features.npy".format(feature_mode, mode))
#         np.save(np.array(labels), "features/{}/{}_labels.npy".format(feature_mode, mode))   
        pickle.dump(features, open("features/{}/{}_features_part{}.pkl".format(feature_mode, mode, f+1), 'wb'))
        pickle.dump(labels, open("features/{}/{}_labels_part{}.pkl".format(feature_mode, mode, f+1), 'wb'))
    
print("Feature extraction complete")

['baby-cry', 'blender', 'door', 'saw', 'alarm-clock', 'cooking', 'chopping', 'toothbrush', 'doorbell', 'phone-ring']
Processing file train baby-cry 6_13805_33854-hq.wav
Processing file train baby-cry 90_179392_3332423-hq.wav
Processing file train baby-cry EFX INT Baby Cry Coo Breathing 05.wav
Processing file train baby-cry BABY- FRETTING.wav
Processing file train baby-cry 21_259607_4841074-hq.wav
Processing file train baby-cry aeroplane ambience 04.wav
Processing file train baby-cry 198411_1648170-hq_chunk25.wav


<ipython-input-2-3064f1c41d0d>:82: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav_data = wavfile.read(wav_file)


Processing file train baby-cry hhM7pEOpEAo.wav
Processing file train baby-cry 198411_1648170-hq_chunk13.wav
Processing file train baby-cry 61262_321967-hq_chunk3.wav
Processing file train baby-cry 198411_1648170-hq_chunk16.wav
Processing file train baby-cry 4 month old baby cry fuss grunt to burble wet .wav
Processing file train baby-cry 15_13802_33854-hq.wav
Processing file train baby-cry tupFX4EzPn8.wav
Processing file train baby-cry 11_211528_2451161-hq.wav
Processing file train baby-cry PK3ex2rqzLE.wav


<ipython-input-2-3064f1c41d0d>:82: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav_data = wavfile.read(wav_file)


Processing file train baby-cry 04 Week Old Girl, Frantic Crying_chunk2.wav
Processing file train baby-cry 33_61262_321967-hq.wav
Processing file train baby-cry 04 Week Old Girl, Frantic Crying_chunk12.wav
Processing file train baby-cry 1.wav
Processing file train baby-cry 06 Week Old Boy, Tired Yelling_chunk1.wav
Processing file train baby-cry 02 Day Old Boy, Crying_chunk10.wav
Processing file train baby-cry 04 Week Old Girl, Frantic Crying_chunk13.wav
Processing file train baby-cry 09 Week Old Boy, Wailing While Being_chunk2.wav
Processing file train baby-cry 12_58178_321967-hq.wav
Processing file train baby-cry 3F5POA5nsAs.wav
Processing file train baby-cry 04 Week Old Girl, Frantic Crying_chunk17.wav
Processing file train baby-cry 5_59579_27178-hq.wav
Processing file train baby-cry 198411_1648170-hq_chunk20.wav
Processing file train baby-cry An upset baby crying- multiple yells- sobs and whines_AOS00169.wav
Processing file train baby-cry 37_80478_556875-hq.wav
Processing file train 

...

Processing file train flush 320332_3667256-hq_chunk0.wav
Processing file train flush 20736_82273-hq_chunk0.wav
Processing file train flush 340295_4724636-hq_chunk1.wav
Processing file train flush 137244_2515465-hq_chunk0.wav
Processing file train flush 58201_382028-hq_chunk1.wav
Processing file train flush 63739_321967-hq_chunk0.wav
Processing file train flush Bathroom_toiletflush.wav
Processing file train flush 346785_6309459-hq_chunk0.wav
Processing file train flush 104953_161750-hq_chunk0.wav
Processing file train flush 67 Urinal Flushing 2_chunk0.wav
Processing file train flush h5IRNNXdpgc.wav
Processing file train flush Hospital bathroom toilet flush_BLASTWAVEFX_11079.wav
Processing file train flush EFX INT Toilet Flush Far 02.wav
Processing file train flush 83497_954389-hq_chunk0.wav
Processing file train flush Toilet bathroom flush clog stop_BLASTWAVEFX_03663.wav
Processing file train flush 341065_6000641-hq_chunk0.wav
Processing file train flush Toilet flushing_AOS00996.wav
Pro

Processing file train flush 323637_5558195-hq_chunk0.wav
Processing file train flush 320320_3667256-hq_chunk0.wav
Processing file train flush 126532_2044671-hq_chunk0.wav
Processing file train flush UjTuzd-jNCI.wav
Processing file train flush Public restroom toilet flush long drain_BLASTWAVEFX_28044.wav
Processing file train flush Jet toilet flush 1.wav
Processing file train flush Toilet flush powerfully with glugs 3.wav
Processing file train flush 98782_1661348-hq_chunk0.wav
Processing file train flush 321729_1998557-hq_chunk0.wav
Processing file train flush 70280_957391-hq_chunk0.wav
Processing file train flush 51170_179538-hq_chunk1.wav
Processing file train flush Flush 1.wav
Processing file train flush 98763_1661348-hq_chunk0.wav
Processing file train flush 213245_1038806-hq_chunk4.wav
Processing file train flush Industrial public toilet flushing_AOS00862.wav
Processing file train flush 82671_1317029-hq_chunk1.wav
Processing file train flush 135441_2415245-hq_chunk1.wav
Processing 

Processing file train flush Yacht toilet flush boat marine bathroom lid down boat_BLASTWAVEFX_31815.wav
Processing file train flush 272012_3364247-hq_chunk1.wav
Processing file train flush 237574_2377160-hq_chunk1.wav
Processing file train flush toilet_flush_lid_down_001.wav
Processing file train flush 398337_7618848-hq_chunk0.wav
Processing file train flush Airplane toilet flush.wav
Processing file train flush 117306_854782-hq_chunk1.wav
Processing file train flush 334607_5863873-hq_chunk1.wav
Processing file train flush 155919_181941-hq_chunk1.wav
Processing file train flush 365540_5407590-hq_chunk0.wav
Processing file train flush bzeDCUNJ3S0.wav
Processing file train flush 588_58-hq_chunk0.wav
Processing file train flush pQVWpEKk6WM.wav
Processing file train flush 232537_3615882-hq_chunk0.wav
Processing file train flush 323637_5558195-hq_chunk1.wav
Processing file train flush niagara.wav
Processing file train flush toilet flush bathroom small canadian.wav
Processing file train flush

Processing file train cat-meow 16 Ten Week Old Kitten, Individual M_chunk0.wav
Processing file train cat-meow wrnmyNXO0-w.wav
Processing file train cat-meow cat crying-Grrrrring.wav
Processing file train cat-meow 13 year old Tabby Cat - Meow-Lunge for Toy.wav
Processing file train cat-meow 6wg3GJZk7Q0.wav
Processing file train cat-meow 28 Tom Cat, Individual Meows_chunk0.wav
Processing file train cat-meow 19278_25498-hq_chunk0.wav
Processing file train cat-meow 0abfTrxjX-4.wav
Processing file train cat-meow 25 Tom Cat, Individual Meows_chunk0.wav
Processing file train cat-meow 66513_334103-hq_chunk0.wav
Processing file train cat-meow uWQI5DCKVdU.wav
Processing file train cat-meow o7jBSQRJzuQ.wav
Processing file train cat-meow 32 Tom Cat, Individual Meows_chunk0.wav
Processing file train cat-meow 5B7j5DIqBJA.wav
Processing file train cat-meow 26 Tom Cat, Individual Meows_chunk0.wav
Processing file train cat-meow 13 year old Tabby Cat - Meow-Mouthy.wav
Processing file train cat-meow 34 T

Processing file train cat-meow 346234_2884593-hq_chunk0.wav
Processing file train cat-meow ANIMAL DOMESTIC FELINE HOUSE CAT LONG MEOW 01.wav
Processing file train cat-meow 60 Siamese, Individual Meows_chunk0.wav
Processing file train cat-meow 366855_6038576-hq_chunk0.wav
Processing file train cat-meow 63 Burmese, Individual Meows_chunk0.wav
Processing file train cat-meow 55149_4948-hq_chunk0.wav
Processing file train cat-meow 85492_48706-hq_chunk0.wav
Processing file train cat-meow 02 Four Week Old Kitten, Individual_chunk0.wav
Processing file train cat-meow -r1H8hBeYmw.wav
Processing file train cat-meow House Cat Meow_01.wav
Processing file train cat-meow 64005_861714-hq_chunk0.wav
Processing file train cat-meow 4MZrJBy7i-I.wav
Processing file train cat-meow 13 year old Tabby Cat - Meow-Irked.wav
Processing file train cat-meow 370160_5996864-hq_chunk0.wav
Processing file train cat-meow 61 Siamese, Individual Meows_chunk0.wav
Processing file train cat-meow lUP2iPc4i3Q.wav
-------------

In [5]:
feature_mode = 'time_seg'

for mode in ['wild']:
    parent_dir = os.path.join('ubicoustics_data', mode)
    label_name_list = os.listdir(parent_dir)
    
    parts = 1
    for f in range(parts):
        ll = len(label_name_list)
        label_list = label_name_list[int(f/parts*ll):int((f+1)/parts*ll)]
        print(label_list)
        features = []
        labels = []
        
        for label_name in label_list:
            class_features = []
            class_labels = []
            sub_dir = os.path.join(parent_dir, label_name)
            files = os.listdir(sub_dir)

            for fn in files:
                if fn.endswith('.wav') and not fn.startswith('.'):
                    print("Processing file", mode, label_name, fn)
                    file = os.path.join(sub_dir, fn)
                    try:
                        feat = wavfile_to_features(file, feature_mode)
                        lbl = [soundtransfer.label[label_name]] * len(feat)
                        assert len(feat) == len(lbl)
                        class_features.append(feat)
                        class_labels.append(lbl)
                    except AssertionError:
                        print('len feat = {} != len lbl = {}'.format(len(feat), len(lbl)))
                    except:
                        print('--- Error while processing')

            if len(class_features)!=0:
                features.append(class_features)
                labels.append(class_labels)
                assert len(class_features) == len(class_labels)
                print('-'*20)
                print(label_name, len(class_features))
                print()

#         np.save(np.array(features), "features/{}/{}_features.npy".format(feature_mode, mode))
#         np.save(np.array(labels), "features/{}/{}_labels.npy".format(feature_mode, mode))   
        pickle.dump(features, open("features/{}/{}_features.pkl".format(feature_mode, mode, f+1), 'wb'))
        pickle.dump(labels, open("features/{}/{}_labels.pkl".format(feature_mode, mode, f+1), 'wb'))
    
print("Feature extraction complete")

['baby-cry', 'blender', 'door', 'saw', 'alarm-clock', 'cooking', 'chopping', 'toothbrush', 'doorbell', 'phone-ring', 'drill', 'dog-bark', 'hammer', 'engine', 'shaver', 'cough', 'hazard-alarm', 'snore', 'car-horn', 'vacuum', 'hair-dryer', 'knock', 'water-running', 'speech', 'typing', 'dishwasher', 'laugh', 'microwave', 'flush', 'cat-meow']
Processing file test baby-cry 39920_215874-hq.wav
Processing file test baby-cry An upset baby crying- multiple yells- sobs and whines_AOS00168.wav
Processing file test baby-cry 80480_556875-hq.wav
Processing file test baby-cry 369798_4469311-hq.wav
Processing file test baby-cry 388172_7227448-hq.wav
Processing file test baby-cry BabyYelp1.wav
Processing file test baby-cry 13805_33854-hq.wav
Processing file test baby-cry 388177_7227448-hq.wav
Processing file test baby-cry 4 month old baby cry stutter .wav
Processing file test baby-cry 406297_612689-hq.wav
Processing file test baby-cry 346663_2104797-hq.wav
Processing file test baby-cry 259607_4841074-h

<ipython-input-3-3064f1c41d0d>:82: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav_data = wavfile.read(wav_file)


Processing file test blender blender juice 01.wav
Processing file test blender Blender starting and running constantly with the lid off then turning off with a glorp 1.wav
Processing file test blender 347682_6321099-hq.wav
Processing file test blender 56563_533680-hq.wav
Processing file test blender 69 Electric Blender Cycle 3.wav
Processing file test blender Beehive retro blender pulsing x2.wav
Processing file test blender A blender blends_AOS02923.wav
Processing file test blender Blender starting and running with many pulses.wav
Processing file test blender Beehive retro blender dry fire without carafe hard.wav
Processing file test blender 65973_321967-hq.wav
Processing file test blender 68 Electric Blender Cycle 2.wav
Processing file test blender servo_stand_blender_08.wav
Processing file test blender 85 Stationary Electric Mixer.wav
Processing file test blender 72 Electric Can Opener.wav
Processing file test blender 71 Electric Blender Cycle 5.wav
Processing file test blender 71792

Processing file test alarm-clock 67033_946237-hq.wav
Processing file test alarm-clock alarm-clock-beep.wav
Processing file test alarm-clock 233645_1752933-hq.wav
Processing file test alarm-clock 321832_2567799-hq.wav
Processing file test alarm-clock 96063_704006-hq.wav
Processing file test alarm-clock Chiming of a grandfather clock_AOS01694.wav
Processing file test alarm-clock 266668_3899133-hq.wav
Processing file test alarm-clock 19 Chimes From A Grandfather Clock S.wav
Processing file test alarm-clock 209536_3915371-hq.wav
Processing file test alarm-clock 84423_1262700-hq.wav
Processing file test alarm-clock Old Alarm Clock Ring 2.wav
Processing file test alarm-clock 190554_2457428-hq.wav
Processing file test alarm-clock Musical time melody clock chimes_BLASTWAVEFX_07014.wav
Processing file test alarm-clock 28 Chimes From A Grandfather Clock S.wav
Processing file test alarm-clock Alarm clock beeps ding 4.wav
Processing file test alarm-clock Clock chimes one o clock_BLASTWAVEFX_27252.

Processing file test toothbrush Brushing teeth_AOS02325.wav
Processing file test toothbrush 272422_5163890-hq.wav
Processing file test toothbrush Brush teeth fast hard_BLASTWAVEFX_13879.wav
Processing file test toothbrush Brush teeth soft_BLASTWAVEFX_13881.wav
Processing file test toothbrush 139331_1779874-hq.wav
Processing file test toothbrush 100024_1531174-hq.wav
Processing file test toothbrush Brush teeth fast_BLASTWAVEFX_25505.wav
Processing file test toothbrush 17092_56518-hq.wav
Processing file test toothbrush Brushing teeth with running water in background_AOS02324.wav
Processing file test toothbrush 325452_5569681-hq.wav
--------------------
toothbrush 25

Processing file test doorbell Electric Bell - To ring the bell -By Augusto Casanovas-.wav
Processing file test doorbell 3-11 Doorbell, Ding-Dong Type, Multi.wav
Processing file test doorbell 223673_3219995-hq.wav
Processing file test doorbell Elevator bell ding ring_BLASTWAVEFX_27950.wav
Processing file test doorbell Buzzer 

...

Processing file test hazard-alarm Alarm Signal.wav
Processing file test hazard-alarm SIREN POLICE AUSTRALIAN.wav
Processing file test hazard-alarm SIREN-AMBULANCE-AUSTRALIAN.wav
Processing file test hazard-alarm EFX EXT Police Siren 03 Doppler By Slow 02.wav
Processing file test hazard-alarm Alarm-Fire-Apartment-Beep-Tone-x2-Stop-Truncated.wav
Processing file test hazard-alarm Alarm hand held crank vintage_BLASTWAVEFX_10843.wav
Processing file test hazard-alarm alarm clock ringing and shut off.wav
Processing file test hazard-alarm fire truck driving by.wav
--------------------
hazard-alarm 50

Processing file test snore Snort-Male-Early Forties-Weak-Slow.wav
Processing file test snore 27404_15220-hq.wav
Processing file test snore 341042_2833408-hq.wav
Processing file test snore 122792_2233594-hq.wav
Processing file test snore 216368_3761494-hq.wav
Processing file test snore Female snorts 3_Nightingale Music Productions_12418.wav
Processing file test snore 352353_2309965-hq.wav
Processi

Processing file test vacuum 194680_3149117-hq.wav
Processing file test vacuum hoover_and_shutdown_001.wav
Processing file test vacuum 249575_4205952-hq.wav
Processing file test vacuum 212059_1979597-hq.wav
Processing file test vacuum 205693_3844637-hq.wav
Processing file test vacuum Vacuum nozzle airy passing movement.wav
Processing file test vacuum 269466_2812229-hq.wav
Processing file test vacuum Vacuum-Bissell-On-Run-Off-Steady-Short 1.wav
Processing file test vacuum Handheld vacuum cleaner piece turn on run turn off cord free_BLASTWAVEFX_24360.wav
Processing file test vacuum 115137_179538-hq.wav
Processing file test vacuum Vacuum cleaner hose attachment clean small debris coins appliance_BLASTWAVEFX_03668.wav
Processing file test vacuum Handheld vacuum cleaner piece turn on run up dirt floor mat cord free_BLASTWAVEFX_24364.wav
Processing file test vacuum 172732_8043-hq.wav
Processing file test vacuum Vacuum cleaner oreck turn on run cement floor turn off cord free electrikbroom app

Processing file test water-running 271473_4190771-hq.wav
Processing file test water-running 262924_4875148-hq.wav
Processing file test water-running 382670_4297074-hq.wav
Processing file test water-running 274071_128404-hq.wav
Processing file test water-running 96700_297142-hq.wav
Processing file test water-running 178080_2977210-hq.wav
Processing file test water-running 85912_1275448-hq.wav
Processing file test water-running 94299_542096-hq.wav
Processing file test water-running 345900_6102266-hq.wav
Processing file test water-running 11563_31600-hq.wav
Processing file test water-running 370012_1189394-hq.wav
Processing file test water-running 1017_1112-hq.wav
Processing file test water-running 201549_3498021-hq.wav
Processing file test water-running 72915_961180-hq.wav
Processing file test water-running 274051_128404-hq.wav
Processing file test water-running 352714_6512973-hq.wav
Processing file test water-running 34072_28216-hq.wav
Processing file test water-running 336518_1738686-h

Processing file test typing 348240_4067257-hq.wav
Processing file test typing 94643_1481393-hq.wav
Processing file test typing 395584_4298084-hq.wav
Processing file test typing 256436_4772965-hq.wav
Processing file test typing 210304_3844637-hq.wav
Processing file test typing 267325_2549998-hq.wav
Processing file test typing 179017_3325164-hq.wav
Processing file test typing 116342_1202376-hq.wav
Processing file test typing 84918_1265412-hq.wav
Processing file test typing 267504_4378549-hq.wav
Processing file test typing 71238_957741-hq.wav
Processing file test typing 241209_55755-hq.wav
Processing file test typing 232981_1159031-hq.wav
Processing file test typing 257788_4157918-hq.wav
Processing file test typing 153444_2635357-hq.wav
Processing file test typing 329052_5284496-hq.wav
Processing file test typing 378073_5966858-hq.wav
Processing file test typing 343426_4092359-hq.wav
Processing file test typing 326291_5569671-hq.wav
Processing file test typing 360502_1738686-hq.wav
Proces

--------------------
microwave 50

Processing file test flush 265366_971447-hq.wav
Processing file test flush 83468_791247-hq.wav
Processing file test flush 100018_1531174-hq.wav
Processing file test flush 235622_4028838-hq.wav
Processing file test flush 389447_6060491-hq.wav
Processing file test flush 264988_4101863-hq.wav
Processing file test flush 334582_5863873-hq.wav
Processing file test flush 39083_146744-hq.wav
Processing file test flush 95218_1202376-hq.wav
Processing file test flush 49538_36697-hq.wav
Processing file test flush 272387_4783367-hq.wav
Processing file test flush 186139_3458104-hq.wav
Processing file test flush 336544_1738686-hq.wav
Processing file test flush 98783_1661348-hq.wav
Processing file test flush 212089_1979597-hq.wav
Processing file test flush 98786_1661348-hq.wav
Processing file test flush 49385_103206-hq.wav
Processing file test flush 125299_296888-hq.wav
Processing file test flush 253542_4616297-hq.wav
Processing file test flush 64461_1198-hq.wav
Pro

In [ ]:
np.concatenate(np.concatenate(features)).shape